In [1]:

import glob
import json
import os
import shutil
import subprocess
import xml.etree.ElementTree as ET
from pathlib import Path
import time
import os
import yaml


import cv2
sys.path.append(os.path.dirname(os.getcwd()))
os.chdir('/Users/seba/local-projects/pedestrians')


from src import data_utils, frame_extract, skeleton_pipeline
from data.jaad.jaad_data import JAAD


In [2]:
jaad_dir = os.path.join('data/jaad')


jaad = JAAD(jaad_dir)

jaad_db = data_utils.JaadDatabase(jaad)
jaad_db.run_database_generator()

jaad_db.add_cropped_bbox()

Database saved to data/processed/jaad_database.pkl


In [3]:
pipeline = skeleton_pipeline.SkeletonPipeline(jaad_db)
pipeline.prepare_images()

Preparing images with the following parameters:
('cropping', True)
('keypoints', True)
('force', False)
Frames for video video_0001 are already extracted.
Cropping frame 568 of 0_1_3b

Cropping frame 69 of 0_1_2b

Commands: ['./build/examples/openpose/openpose.bin', '--image_dir', '/Users/seba/local-projects/pedestrians/data/processed/cropped/video_0001/0_1_3b', '--write_json', '/Users/seba/local-projects/pedestrians/data/processed/keypoints/video_0001/json/0_1_3b', '--write_images', '/Users/seba/local-projects/pedestrians/data/processed/keypoints/video_0001/images/0_1_3b', '--model_pose', 'BODY_25B', '--net_resolution', '-1x1280', '--net_resolution_dynamic', '0', '--scale_number', '4', '--scale_gap', '0.25', '--display', '0']
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...


KeyboardInterrupt: 